In [ ]:
from __future__ import annotations

import numpy as np
from matplotlib import pyplot as plt

%env SPDLOG_LEVEL=trace
import mcerl
from mcerl.env import Env


In [ ]:
test_map = np.ndarray(shape=(100, 100), dtype=np.uint8)
test_map.fill(255)
test_map[20:70, 20:30] = 0
test_map[30:40, 10:50] = 0
test_grid_map = mcerl.GridMap(test_map)
num_agents = 2
agent_poses = [(10, 10), (60, 60)]
num_rays = 32
max_steps = 200
max_steps_per_agent = 40
ray_range = 20
velocity = 1
min_frontier_size = 5
max_frontier_size = 20
env = Env(
    num_agents=num_agents,
    max_steps=max_steps,
    max_steps_per_agent=max_steps_per_agent,
    velocity=velocity,
    sensor_range=ray_range,
    num_rays=num_rays,
    min_frontier_pixel=min_frontier_size,
    max_frontier_pixel=max_frontier_size,
)

In [ ]:
rollout = []
frame_data = env.reset(test_map, agent_poses)
rollout.append(frame_data)
while True:
    frame_data = env.step(*env.random_action())
    rollout.append(frame_data)
    if env.done() is True:
        break

In [ ]:
rollout

In [ ]:
plt.imshow(env._env.agent_map(1), cmap="gray",vmin=0, vmax=255)

In [ ]:
# split trajectory into agent-wise trajectories

def split_trajectories(rollout):
    agent_rollouts = []
    for agent_id in range(num_agents):
        agent_rollout=[frame_data for frame_data in rollout if frame_data["robot_id"] == agent_id]
        agent_rollouts.append(agent_rollout)
    return agent_rollouts

In [ ]:
from tensordict import LazyStackedTensorDict
import tensordict


def trajectory_to_tensordict(traj):
    """
    Note: this will change the order of frontier points from (n,x,y) to (x,y,n)
    """
    return LazyStackedTensorDict.maybe_dense_stack(
        [tensordict.TensorDict(frame_data).transpose(0, 1) for frame_data in traj]
    )

In [ ]:
rollout = split_trajectories(rollout)
rollout

In [ ]:
for agent_rollout in rollout:
    agent_rollout=trajectory_to_tensordict(agent_rollout)